# 文本提取

In [1]:
import fitz  # PyMuPDF
import pdb
import pandas as pd 
import os
from tqdm import tqdm

In [2]:
# 建立文本存储dataframe
text_df = pd.DataFrame(columns=["PDF_name", "page", "p_index",
                                "content", "center_x", "center_y"])

## 函数定义

In [3]:
def paragraph(file_path, page_number, y_threshold = 11.5, text_df = text_df):
    # 打开PDF文件
    pdf_file = fitz.open(file_path)

    # 获取页面
    page = pdf_file[page_number]
    # 获取页面高度
    page_height = page.rect.height

    # 获取页面上的所有文本块及其位置
    blocks = page.get_text("blocks")

    # 初始的段落, 坐标
    paragraphs = []
    coordinates = []

    # 初始的文本块
    current_text = ""
    x0, y0, x1, y1 = blocks[0][:4]

    # 遍历每个文本块
    for block in blocks:
        bx0, by0, bx1, by1, text, _, _ = block
            
        # 忽略图像信息块
        if "<image:" in text:
            continue
        
        # y轴阈值
        y_close_enough = abs(by0 - y1) < y_threshold
        
        # 检查y坐标是否相邻以确定是否在同一段落中
        if y_close_enough:
            current_text +=  text.strip()
            x1 = max(x1, bx1)
            y1 = max(y1, by1)
        else:
            # 保存段落和计算中心坐标
            paragraphs.append(current_text.replace(" ",""))
            coordinates.append(((x0 + x1) / 2, (y0 + y1) / 2))

            # 开始新的文本块
            current_text = text.strip()
            x0, y0, x1, y1 = bx0, by0, bx1, by1

    # 保存最后一个段落和中心坐标
    paragraphs.append(current_text.replace(" ",""))
    coordinates.append(((x0 + x1) / 2, (y0 + y1) / 2))

    # 遍历段落和中心坐标
    j = 0
    for para, coord in zip(paragraphs, coordinates):
#         print(f"段落: {para}\n中心: {coord}\n")
        
        # 保存至dataframe
        center_x, center_y = coord
        
        # 判断是不是页眉页脚
        if not is_header_or_footer(center_x, center_y, page_height, header_footer_threshold=100):
        
            info = pd.Series(["",page_number + 1 ,j ,para, center_x, center_y],
                            index=["PDF_name", "page", "p_index",
                                    "content", "center_x", "center_y"])
            text_df = pd.concat([text_df, pd.DataFrame([info])], ignore_index=True)
        j = j + 1
        
    # 关闭PDF文件
    pdf_file.close()
    
    return text_df

In [4]:
def get_bundle_path(directory):
    files = [f for f in os.listdir(directory) if f != '.DS_Store']
    return [os.path.join(directory, file) for file in files if os.path.isfile(os.path.join(directory, file))]

In [5]:
def PDF_text_extract(file_path, text_df=text_df):
    # 打开PDF并获取名称
    pdf_file = fitz.open(file_path)
    pdf_file_name = os.path.basename(file_path)
        
    for page_number in range(len(pdf_file)):
        # 提取当页PDF文本内容
        try:
            text_df = pd.concat([text_df, paragraph(file_path, page_number)], ignore_index=True)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            
    text_df.PDF_name = pdf_file_name
    return text_df[text_df["content"] != ""]

## 排除页眉页脚

In [6]:
def is_header_or_footer(center_x, center_y, page_height, header_footer_threshold):
    
    # 检查位置是否在页眉或页脚的阈值范围内
    if center_y < header_footer_threshold or center_y > (page_height - header_footer_threshold):
        return True

    return False

## 修改输入输出路径

In [7]:
# 输入文件夹路径
file_bundle = get_bundle_path("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2021")

# 输出csv文件路径
file_output_path ="/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/ESG 2021/01 Text"

## 运行main

In [8]:
for file in tqdm(file_bundle):
    text_df = pd.concat([text_df, PDF_text_extract(file)], ignore_index=True)

  8%|███▎                                      | 19/238 [00:52<07:04,  1.94s/it]

Error occurred: list index out of range


 15%|██████▏                                   | 35/238 [01:44<09:23,  2.77s/it]

Error occurred: list index out of range


 16%|██████▌                                   | 37/238 [01:48<07:53,  2.36s/it]

...

 95%|███████████████████████████████████████  | 227/238 [10:51<00:30,  2.74s/it]

Error occurred: list index out of range


 99%|████████████████████████████████████████▋| 236/238 [11:09<00:03,  1.75s/it]

Error occurred: list index out of range


100%|█████████████████████████████████████████| 238/238 [11:17<00:00,  2.85s/it]


In [9]:
# try:
# #     pdb.set_trace()
#     text_df = paragraph("./Playground/EsgReport_PDF/00998.HK-中信银行-中信银行 中信银行股份有限公司2022年度可持续发展报告-2023-03-24.pdf", 
#               7)
# except Exception as e:
#     print(f"An error occurred: {str(e)}")

In [10]:
text_df

,PDF_name,page,p_index,content,center_x,center_y
0,600350.SH-山东高速-山东高速股份有限公司2021年环境、社会及治理报告-2022-...,2,2,关于本报告,169.251999,171.902611
1,600350.SH-山东高速-山东高速股份有限公司2021年环境、社会及治理报告-2022-...,2,3,本报告内容时间界限为2021年1月1日至2021年12月31日，部分内容超出上述范围。,273.216419,456.935013
2,600350.SH-山东高速-山东高速股份有限公司2021年环境、社会及治理报告-2022-...,2,4,本报告覆盖山东高速股份有限公司及附属公司。,188.751896,542.731995
3,600350.SH-山东高速-山东高速股份有限公司2021年环境、社会及治理报告-2022-...,2,5,全部信息数据来自山东高速股份有限公司所属各部门、各单位的正式文件、统计报告与财务报告，以及经...,296.574604,654.926025
4,600350.SH-山东高速-山东高速股份有限公司2021年环境、社会及治理报告-2022-...,2,6,本报告参照上海证券交易所《公司履行社会责任的报告编制指引》进行编写，并适度参考中国社科院发布...,895.254578,474.629013
...,...,...,...,...,...,...
148839,688248.SH-南网科技-南方电网电力科技股份有限公司2021年度环境、社会及管治(ES...,32,10,很好\n较好\n一般\n较差\n差,171.636448,429.617096
148840,688248.SH-南网科技-南方电网电力科技股份有限公司2021年度环境、社会及管治(ES...,32,11,很好\n较好\n一般\n较差\n差,171.636448,469.325623
148841,688248.SH-南网科技-南方电网电力科技股份有限公司2021年度环境、社会及管治(ES...,32,12,3．您认为本报告是否能反映南网科技对经济、社会和环境的重大影响？,170.505199,412.193832
148842,688248.SH-南网科技-南方电网电力科技股份有限公司2021年度环境、社会及管治(ES...,32,13,4．您认为报告的可读性，即报告的逻辑主线、内容设计、语言文字和版式设计？,181.755199,451.973831


## 处理文本

In [11]:
def taboo_content(t):
    taboo = ['案例']
    return t in taboo

mask = text_df['content'].apply(taboo_content)
text_df = text_df[~mask]

In [12]:
def is_all_digits(s):
    return s.isdigit()

mask = text_df["content"].apply(is_all_digits)

text_df = text_df[~mask]

In [13]:
regex = r'[\n]'

# 使用 str.replace 方法，将这些字符替换为空字符串
# 这会剔除这些字符，而保留其他文本
text_df["content"] = text_df["content"].str.replace(regex, '', regex=True)

## 保存

In [14]:
# 检查输出目录是否存在，如果不存在则创建
if not os.path.exists(file_output_path):
    os.makedirs(file_output_path)
    
text_df.to_excel(os.path.join(file_output_path, "text_df_2.xlsx"), index=False)

In [15]:
text_df

,PDF_name,page,p_index,content,center_x,center_y
0,600350.SH-山东高速-山东高速股份有限公司2021年环境、社会及治理报告-2022-...,2,2,关于本报告,169.251999,171.902611
1,600350.SH-山东高速-山东高速股份有限公司2021年环境、社会及治理报告-2022-...,2,3,本报告内容时间界限为2021年1月1日至2021年12月31日，部分内容超出上述范围。,273.216419,456.935013
2,600350.SH-山东高速-山东高速股份有限公司2021年环境、社会及治理报告-2022-...,2,4,本报告覆盖山东高速股份有限公司及附属公司。,188.751896,542.731995
3,600350.SH-山东高速-山东高速股份有限公司2021年环境、社会及治理报告-2022-...,2,5,全部信息数据来自山东高速股份有限公司所属各部门、各单位的正式文件、统计报告与财务报告，以及经...,296.574604,654.926025
4,600350.SH-山东高速-山东高速股份有限公司2021年环境、社会及治理报告-2022-...,2,6,本报告参照上海证券交易所《公司履行社会责任的报告编制指引》进行编写，并适度参考中国社科院发布...,895.254578,474.629013
...,...,...,...,...,...,...
148839,688248.SH-南网科技-南方电网电力科技股份有限公司2021年度环境、社会及管治(ES...,32,10,很好较好一般较差差,171.636448,429.617096
148840,688248.SH-南网科技-南方电网电力科技股份有限公司2021年度环境、社会及管治(ES...,32,11,很好较好一般较差差,171.636448,469.325623
148841,688248.SH-南网科技-南方电网电力科技股份有限公司2021年度环境、社会及管治(ES...,32,12,3．您认为本报告是否能反映南网科技对经济、社会和环境的重大影响？,170.505199,412.193832
148842,688248.SH-南网科技-南方电网电力科技股份有限公司2021年度环境、社会及管治(ES...,32,13,4．您认为报告的可读性，即报告的逻辑主线、内容设计、语言文字和版式设计？,181.755199,451.973831
